In [ ]:
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install nltk

In [1]:

import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import regex as re
from sklearn.metrics import *
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import string
import operator
import os
import requests

import nltk
nltk.download('stopwords') # If needed
nltk.download('punkt') # If needed
nltk.download('wordnet') # If needed
# nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Importing data from drive

This cell tries to connect to google drive if you click on cancel it will pick the csv file from git repo

In [2]:

try: 

    from google.colab import drive

    # Mount colab to google drive need to unconnect if we want to connect from google drive
    # drive.mount("/content/gdrive")


    #List of categories
    categories = ["business", "entertainment", "politics", "sport", "tech"]

    #Path in google drive
    path = "/content/gdrive/MyDrive/Colab Notebooks/text_cl/bbc/"

    total_data = []

    #This loop iterates every folder and stores text content in total_data variable
    for category in categories:
        file_paths = os.listdir(path+category)
        for name in file_paths:
            file_name = "{0}{1}/{2}".format(path,category,name)
            with open(file_name,'rb') as file:
                text = file.read().decode(errors='replace').replace("\n"," ")
            total_data.append((category,name,text))

  
    # Creating the dataframe which contains all the text content
    df = pd.DataFrame(total_data)


except :
  print("Drive is not mounted so picking data from github url")
  df = pd.read_csv("https://raw.githubusercontent.com/sunnybandlamudi/AML/master/bbc.csv")  

# Adding names to data columns
df.columns=["category","file_name","text"]


# Displaying the total records present in each category
df['category'].value_counts()



Drive is not mounted so picking data from github url


sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

# Importing data from Git repository

Stored all text content to csv file and uploaded to git repository.

Github Link :https://raw.githubusercontent.com/sunnybandlamudi/AML/master/bbc.csv



In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/sunnybandlamudi/AML/master/bbc.csv") 

# Displaying the data frame object


In [4]:
#Using to display data frame
df

,category,file_name,text
0,business,289.txt,UK economy facing 'major risks' The UK manufa...
1,business,504.txt,Aids and climate top Davos agenda Climate cha...
2,business,262.txt,Asian quake hits European shares Shares in Eu...
3,business,276.txt,India power shares jump on debut Shares in In...
4,business,510.txt,Lacroix label bought by US firm Luxury goods ...
...,...,...,...
2220,tech,086.txt,Warning over Windows Word files Writing a Mic...
2221,tech,253.txt,Fast lifts rise into record books Two high-sp...
2222,tech,247.txt,Nintendo adds media playing to DS Nintendo is...
2223,tech,290.txt,Fast moving phone viruses appear Security fir...


# Text prepossing

`text_process` fucntion takes the text content as the input and returns the processed text.

Creating a new `text_clean` column to stote the processed text.

In [5]:

#lemmatizer object
lemmatizer = nltk.stem.WordNetLemmatizer()

#set of english stop words
stopwords=set(nltk.corpus.stopwords.words('english'))

#Adding frequent characters to stopwords list
stopwords.add(".")
stopwords.add(",")
stopwords.add("--")
stopwords.add("``")

# Function to returns the text of tokens from a string_data
def text_process(string_data): 

    # separating every sentence present in string_data
    sentence_split=nltk.tokenize.sent_tokenize(string_data)
    
    list_tokens=[]

    #Iterating through every sentence
    for sentence in sentence_split:

        # Below line removes the puntuation 
        #ex:- "The Government" converts into The Goverment(without quotation marks)
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))

        # Using word_tokenize function to tokenize the sentence. list_tokens_sentence contains all the token present in sentence variable
        list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)

        # Iterating through all the tokens
        for token in list_tokens_sentence:
            # If token present in stopwords set we are not adding it ot list_tokens
            if token in stopwords:
              continue;
            
            # we are lemmatizeing token, converting to lower case and adding it to list_tokens 
            list_tokens.append(lemmatizer.lemmatize(token).lower())

    # return concatenated string of all the tokens present in string_data
    return " ".join(list_tokens)


# creating the new column text_clean which contains the processed string(Tokenization, Lemmatization, Stopwords removal)
df['text_clean'] = df['text'].apply(lambda x: text_process(x))

print("Cleaning done")




Cleaning done


# Displaying dataframe after text processing

`text_clean` column contains the processed text

In [6]:
# Displaying the dataframe which contains the processed text
df

,category,file_name,text,text_clean
0,business,289.txt,UK economy facing 'major risks' The UK manufa...,uk economy facing major risk the uk manufactur...
1,business,504.txt,Aids and climate top Davos agenda Climate cha...,aids climate top davos agenda climate change f...
2,business,262.txt,Asian quake hits European shares Shares in Eu...,asian quake hit european share shares europes ...
3,business,276.txt,India power shares jump on debut Shares in In...,india power share jump debut shares indias lar...
4,business,510.txt,Lacroix label bought by US firm Luxury goods ...,lacroix label bought us firm luxury good group...
...,...,...,...,...
2220,tech,086.txt,Warning over Windows Word files Writing a Mic...,warning windows word file writing microsoft wo...
2221,tech,253.txt,Fast lifts rise into record books Two high-sp...,fast lift rise record book two highspeed lift ...
2222,tech,247.txt,Nintendo adds media playing to DS Nintendo is...,nintendo add medium playing ds nintendo releas...
2223,tech,290.txt,Fast moving phone viruses appear Security fir...,fast moving phone virus appear security firm w...


# Separating training, development and testing dataset

`shuffle` function is used to shuffle the data present in dataframe


we are splitting data in the ration **80:10:10**

**80%** Train set

**10%** Development set

**10%** Test

---

X_train_text  = Contains all the processed text from train set 

Y_train_label = Contains target labels from train test 

 

X_dev_text = Contains all the processed text from dev set 

Y_dev_Label = Contains target labels from dev set 

 

X_test_text = Constians all the processed text from test set 

Y_test_label = Contains target labels from test set 

In [18]:

from sklearn.utils import shuffle

# shuffiling the whole dataframe
df = shuffle(df,random_state=11)


# considering 80% as the train size
train_size = int(len(df)*0.8)

# considering 10% as the development size
dev_size = int(len(df)*0.1)

# considering 10% as the test size
test_size = int(len(df)*0.1)


# iloc is used to slice the rows [0:80%]
train_set = df.iloc[:train_size]

# Assigning next 10% to dev_set variable [80% : 90%] 
dev_set = df.iloc[train_size: train_size + dev_size]

# Assigning remaining records to test_set variable [90% : 100%]
test_set = df.iloc[ train_size + dev_size: ]



print("Shape of Train Set is ",train_set.shape)

print("Shape of Dev Set is ",dev_set.shape)

print("Shape of Test Set is ",test_set.shape)


# Separating the training features
X_train_text = train_set["text_clean"]
# Separating the target labels
Y_train_label = train_set["category"]

# Separating the dev features
X_dev_text = dev_set["text_clean"]
# Separating the dev target labels
Y_dev_label = dev_set["category"]

# Separating the test features
X_test_text = test_set["text_clean"]
# Separating the test target labels
Y_test_label = test_set["category"]



Shape of Train Set is  (1780, 4)
Shape of Dev Set is  (222, 4)
Shape of Test Set is  (223, 4)


`combine_feature` function is a utility function which combines the vector horizontally 

Example:

a = `[[0,2],[2,3]]`

b = `[[3,4],[5,6]]`

combine_feature returns = `[[0,2,3,4],[2,3,5,6]`

In [12]:
# Utility function which combines vectors
def combine_features(*vectors):      
    return np.hstack(vectors)


# Model Training

we are using SVC as the machine learning model.

**Feature Selection Technique:** we are using `CountVectorizer` `TfidVectorizer(Unigram)` `TfidVectorizer(Bigram)` to convert text content to vectors and using `SelectKBest`, `chi2` to reduce the dimensionality.


In this technique we are selecting k best best features for `CountVectorizer` `TfidVectorizer(Unigram)` `TfidVectorizer(Bigram)` and combining those features to one feature vector.

**Example:**

**CountVectorizer** generated some 10000 feature. By using `chi2` and `selectKBest` we are picking 500 best feature vectors.

**TFidVectorizer(Unigram)** generated some 9000 feature. By using `chi2` and `selectKBest` we are picking 500 best feature vectors.

**TFidVectorizer(Bigram)** generated some 5000 feature. By using `chi2` and `selectKBest` we are picking 500 best feature vectors.

we combine all those best features into one single featuer vector of size 1500

we will train our algorithm with these 1500 features.

**HyperTuning**: 



In [19]:

# One of the feature selection technique is based on word frequency. 
count_vectorizer = CountVectorizer(ngram_range= (1,1),min_df =5)

# Creating a vector object that does unigram conversion
unigram_vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', min_df =3, ngram_range= (1,1))

# Creating a bigram object that does bigram conversion
bigram_vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', min_df =5, ngram_range= (2,2))


model_svc_classifier = sklearn.svm.SVC(kernel="linear",gamma='auto')


# Hyper tuning parametes by tuning the vector values
k_best = [100,500, 1000 ,2000]

# Iterating through all the objects
for k in k_best:

      print(f"Training with SVC as the model")
      print("------------------------------------------")

      # Creating objects which picks k best features
      count_best = SelectKBest(chi2, k=k)
      unigram_best = SelectKBest(chi2, k=k)
      bigram_best = SelectKBest(chi2, k=k)


      # Converting train text features to vectors features - Count Vectorizer
      count_vector_features = count_vectorizer.fit_transform(X_train_text).toarray()
      # Below line select the K best features
      count_best_features= count_best.fit_transform(count_vector_features, Y_train_label)
      print(f"Count Vectorizer: \n\t Reducing features from {count_vector_features.shape} to {count_best_features.shape} using chi2")

      # Converting train text features to vectors features - TFid Vectorizer unigram
      unigram_vector_features = unigram_vectorizer.fit_transform(X_train_text).toarray()
      # Below line select the K best features
      unigram_best_features = unigram_best.fit_transform(unigram_vector_features, Y_train_label)
      print(f"Tfid Vectorizer(Uni-gram): \n\t Reducing features from {unigram_vector_features.shape} to {unigram_best_features.shape} using chi2")
      
      # Converting train text features to vectors features - TFid Vectorizer bigram
      bigram_vector_features = bigram_vectorizer.fit_transform(X_train_text).toarray()
      # Below line select the K best features
      bigram_best_features = bigram_best.fit_transform(bigram_vector_features, Y_train_label)
      print(f"Tfid Vectorizer(Bi-gram): \n\t Reducing features from {bigram_vector_features.shape} to {bigram_best_features.shape} using chi2 \n")


      # Combining all the best features selected
      X_train_combined_vectors = combine_features(count_best_features,unigram_best_features,bigram_best_features)

     
      print (f"Combined features are of shape {str(X_train_combined_vectors.shape)}\n")


      # training our model with fit method using train dataset
      model_svc_classifier.fit(X_train_combined_vectors,Y_train_label)

      
      #----------------------------------Transforming Dev data set--------------------------------------------


    
      # Transforming dev set to vectors
      count_vector_features = count_vectorizer.transform(X_dev_text).toarray()
      # Selecting k best features dev set
      count_best_features = count_best.transform(count_vector_features)

      # Transforming dev set to vectors
      unigram_vector_features = unigram_vectorizer.transform(X_dev_text).toarray()
      # Selecting k best features dev set
      unigram_best_features = unigram_best.transform(unigram_vector_features)

    
      # Transforming dev set to vectors
      bigram_vector_features = bigram_vectorizer.transform(X_dev_text).toarray()
      # Selecting k best features dev set
      bigram_best_features = bigram_best.transform(bigram_vector_features)



      # Combining all the best featues selected
      X_dev_combined_vectors = combine_features(count_best_features,unigram_best_features,bigram_best_features)

      
      #------------------------------------Validation of DEV data set------------------------------------
      
      # predicting the output of dev data set
      Y_pred_dev =  model_svc_classifier.predict(X_dev_combined_vectors)

      accuracy_dev = accuracy_score(Y_dev_label,Y_pred_dev)
     
      print(f"Results for SVM on DEV set is with k = {k} ")
      print("-----------------------------------------")

      precision=precision_score(Y_pred_dev, Y_dev_label, average='macro')
      recall=recall_score(Y_pred_dev, Y_dev_label, average='macro')
      f1=f1_score(Y_pred_dev, Y_dev_label, average='macro')
      accuracy=accuracy_score(Y_pred_dev, Y_dev_label)

      print ("Precision: "+str(round(precision,3)))
      print ("Recall: "+str(round(recall,3)))
      print ("F1-Score: "+str(round(f1,3)))
      print ("Accuracy: "+str(round(accuracy,3))+'\n\n')




Training with SVC as the model
------------------------------------------
Count Vectorizer: 
	 Reducing features from (1780, 7835) to (1780, 100) using chi2
Tfid Vectorizer(Uni-gram): 
	 Reducing features from (1780, 11377) to (1780, 100) using chi2
Tfid Vectorizer(Bi-gram): 
	 Reducing features from (1780, 5466) to (1780, 100) using chi2 

Combined features are of shape (1780, 300)

Results for SVM on DEV set is with k = 100 
-----------------------------------------
Precision: 0.963
Recall: 0.965
F1-Score: 0.964
Accuracy: 0.964


Training with SVC as the model
------------------------------------------
Count Vectorizer: 
	 Reducing features from (1780, 7835) to (1780, 500) using chi2
Tfid Vectorizer(Uni-gram): 
	 Reducing features from (1780, 11377) to (1780, 500) using chi2
Tfid Vectorizer(Bi-gram): 
	 Reducing features from (1780, 5466) to (1780, 500) using chi2 

Combined features are of shape (1780, 1500)

Results for SVM on DEV set is with k = 500 
------------------------------

#Testing our model on Test data set


we are getting good accuray for k == 2000

In [20]:



#-----------Transforming Test Data set---------------

count_vector_features = count_vectorizer.transform(X_test_text).toarray()
count_best_features = count_best.transform(count_vector_features)

unigram_vector_features = unigram_vectorizer.transform(X_test_text).toarray()
unigram_best_features = unigram_best.transform(unigram_vector_features)

bigram_vector_features = bigram_vectorizer.transform(X_test_text).toarray()
bigram_best_features = bigram_best.transform(bigram_vector_features)



# Transforming test text features to vectors
X_test_combined_vectors = combine_features(count_best_features,unigram_best_features,bigram_best_features)

# predicting the output of test data set
Y_pred_test =  model_svc_classifier.predict(X_test_combined_vectors)


print(f"Results for SVM on Test set is")
print("-----------------------------------------")

precision=precision_score(Y_pred_test, Y_test_label, average='macro')
recall=recall_score(Y_pred_test, Y_test_label, average='macro')
f1=f1_score(Y_pred_test, Y_test_label, average='macro')
accuracy=accuracy_score(Y_pred_test, Y_test_label)

print ("Precision: "+str(round(precision,3)))
print ("Recall: "+str(round(recall,3)))
print ("F1-Score: "+str(round(f1,3)))
print ("Accuracy: "+str(round(accuracy,3))+'\n\n')


Results for SVM on Test set is
-----------------------------------------
Precision: 0.965
Recall: 0.966
F1-Score: 0.965
Accuracy: 0.964


